**Домашняя работа №3**

Брылёва Екатерина МБД191

In [56]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from scipy.integrate import odeint
from mpl_toolkits import mplot3d

**1. Химический Ходжкин-Хаксли**

---



1.1 Нейроны

Даны три нейрона: A, B, C.

Нейрон A: тормозящий GABA, нейрон B: возбуждабщий Glu, нейрон C: пирамидный. Состояние нейрона C полностью зависит от работы синапсов Sac, Sbc. 

Природа спайкинговой активности нейрона A - вероятностная(распр. - пуассоновское). Природа спайкинговой активности нейрона B - вероятностная(распр. - равномерное).

Модель нейрона C - модель Ходжкина-Хаксли.

1.2 Синапсы

Каждый синапс оперирует следующими величинами:

 - количество поступающего нейромедиатора Lincoming от пресинаптического нейрона (в молях)
 - количество убывающего нейромедиатора Loutcoming из синапса (в молях)
 - объем синапса V
 - количество рецепторов Rsac на постсинаптической мембране в синапсе Sac и количество рецепторов Rsbc на постсинаптической мембране в синапсе Sbc

In [ ]:
Lincoming_A = 3
Lincoming_B = 3
NA = 1 #количество рецепторов в синапсе Sac
VA = 5 #объем синапса Sac
NB = 1 #количество рецепторов в синапсе Sbc
VB = 5 #объем синапса Sbc
Loutcoming_A = 2
Loutcoming_B = 2

1.3 Рецепторы

Используются рецепторы к глутамату AMPA(проводящие Na+) и рецепторы к гамма-аминомаслянной кислоте GABAa(проводящие Cl-). Вероятность открытия рецептора эквивалентна вероятности связывания рецептора с лигандом. 

1.4 Константы

In [57]:
# Равновесный потенциал ионного канала калия (mS/mm^2)
gK = 0.36
# Равновесный потенциал ионного канала натрия (mS/mm^2)
gNa = 1.2
# Равновесный потенциал ионного канала утечки (mS/mm^2)
gL = 0.003
# Емкость мембраны (nF/mm^2)
Cm = 10.0
# Калиевый потенциал мембраны клетки (mV)
EK = -77.0
# Натриевый потенциал мембраны клетки (mV)
ENa = 55.0
# Хлорный потенциал мембраны клетки (mV)
ECl = -65.0
# Потенциал утечки мембраны клетки (mV)
El = - 54.402
# Начальный потенциал мембраны клетки
V0 = -65.0

In [58]:
KB = 500
KA = 128
gam_B = 10 #константа проводимости рецептора Sbc
gam_A = 8 #константа проводимости рецептора Sac

Модель Ходжкина-Хаксли

Уравнения для расчета активационных (n и m) и инактивационных (h) переменных каналов

In [59]:
# Функции для ионного канала калия

def alpha_n(Vm):
    return (0.01 * (Vm + 55.0)) / (1.0 - np.exp( - 0.1 * (Vm + 55.0)))

def beta_n(Vm):
    return 0.125 * np.exp(- 0.0125 * (Vm + 65.0))

In [60]:
# Функции для ионного канала натрия

def alpha_m(Vm):
    return (0.1 * (Vm + 40.0)) / (1.0 - (np.exp( -0.1 * (Vm + 40))))

def beta_m(Vm):
    return 4.0 * np.exp(- 0.0556 * (Vm + 65.0))

def alpha_h(Vm):
    return 0.07 * np.exp(- 0.05 * (Vm + 65.0))

def beta_h(Vm):
    return 1.0 / (np.exp( - 0.1 * (Vm + 35.0)) + 1.0)

Уравнения для расчета переменных n, m и h

In [61]:
def n_inf(Vm=V0):
    return alpha_n(Vm) / (alpha_n(Vm) + beta_n(Vm))

def m_inf(Vm=V0):
    return alpha_m(Vm) / (alpha_m(Vm) + beta_m(Vm))

def h_inf(Vm=V0):
    return alpha_h(Vm) / (alpha_h(Vm) + beta_h(Vm))

Считаем производные модели

In [70]:
def derivatives(y, t0):
    dy = np.zeros((4,))
    
    Vm = y[0]
    n = y[1]
    m = y[2]
    h = y[3]
    
    GL = (gL / Cm * 1000)
    GK = (gK / Cm * 1000) * np.power(n, 4.0)
    GNa = (gNa / Cm * 1000) * np.power(m, 3.0) * h

    #Движущая сила:
    Vdf_A = Vm - ECl #формула 2
    Vdf_B = Vm - ENa #формула 2

    #Вероятность открытия рецептора:
    Pbound_A = (1 / (1 + KA * VA / (L(t0, "A")))) #формула 3
    Pbound_B = (1 / (1 + KB * VB / (L(t0, "B")))) #формула 3

    #Ток в синапсе:
    Lsynaptic_A = NA * Pbound_A * Vdf_A * gam_A #формула 1
    Lsynaptic_B = NB * Pbound_B * Vdf_B * gam_B #формула 1

    dy[0] =  (Lsynaptic_A + Lsynaptic_B) / Cm * 10 - (GK * (Vm - EK)) - (GNa * (Vm - ENa))
    
    dy[1] = (alpha_n(Vm) * (1.0 - n)) - (beta_n(Vm) * n)
    
    dy[2] = (alpha_m(Vm) * (1.0 - m)) - (beta_m(Vm) * m)
    
    dy[3] = (alpha_h(Vm) * (1.0 - h)) - (beta_h(Vm) * h)
    
    
    return dy

In [71]:
Y = np.array([V0, n_inf(), m_inf(), h_inf()])

**2. Задание**

---



В модели Ходжкина-Хаксли заменить компоненту утечки на сумму токов из возбуждающего и тормозящего синапсов. Балансировать следующие величины: объем синапса, количество поступающего и убывающего нейромедиатора, количество рецепторов в синапсе, форму распределений вероятностей. Если пирамидный нейрон будет спайкаться более 20 секунд подряд с частотой 10 Гц, он погибнет. Если частота спайков превысит 16 Гц на протяжении более 5 секунд, пирамидный нейрон не выживет.

---
1. Котенок должен выжить.
2. добиться устойчивой картины активности пирамидного нейрона(спайки не реже 1 раза в 500 мс).
3. Добиться бёрстинговой активности нейрона (частота не менее 10Гц) не менее 5 раз за время эксперимента.
4. Добиться бёрстинговой активности нейрона (частота не менее 15Гц) не менее 3 раз за время эксперимента.


In [69]:
def L(t0, syn):
  if syn == "A":
    return L_A_d.get(float(t0))
  elif syn == "B":
    return L_B_d.get(float(t0))

In [72]:
T = np.linspace(0.0, 60.0, 10000)

In [73]:
u = 0.5 
p = 3 
L_A = np.zeros(len(T))
for i in range(1, len(T)):
    is_spiking = np.random.poisson(p) > p
    L_A[i] = max(L_A[i - 1] + int(is_spiking) * Lincoming_A - Loutcoming_A, 0.0)
L_A_d = dict(zip(T, L_A))

L_B = np.zeros(len(T))
for i in range(1, len(T)):
    is_spiking = np.random.uniform(0,1) > u
    L_B[i] = max(L_B[i - 1] + int(is_spiking) * Lincoming_B - Loutcoming_B, 0.0)
L_B_d = dict(zip(T, L_B))

In [74]:
Vy = odeint(derivatives, Y, T)

TypeError: ignored

**3. Анализ**

---
Построить графики для ситуации, когда котенок выживет:


1. Зависимость Vm от времени, зависимость n,h,m от времени, зависимость Isynaptic от времени. Сделайте вывод о зависимости амплитуды и частоты Isynaptic и амплитуды и частоты Vm.
 

2. Для каждого промежутка симуляции постройте графики фазовых пространств Vm(t) к n(t), Vm(t) к m(t), Vm(t) к h(t) (3 штуки). Для каждого ли промежутка симуляции существуют предельные циклы каждого фазового пространства?

3. Динамика токов по калию, натрию, хлору. Насколько велика роль калия в этой модели?

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize = (20,15))
ax1.plot(T, Vy[:, 0])
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Vm (mV)')
ax1.set_title('Зависимость Vm от времени')
plt.grid()

ax2.plot(T, Vy[:, 1], label = 'n')
ax2.plot(T, Vy[:, 2], label = 'm')
ax2.plot(T, Vy[:, 3], label = 'h')
ax2.legend()
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel('n, m, h')
ax2.set_title('Зависимость n, h, m от времени')
plt.grid()

ax3.plot(T, Idv)
ax3.set_xlabel('Time (ms)')
ax3.set_ylabel(r'Current density (uA/$mm^2$)')
ax3.set_title('Зависимость I от времени')
plt.grid()

**4. Результат**

---

